1.
Давайте уточним правило трёх сигм. Утверждение: 99.7% вероятностной массы случайной величины $X\sim N\left(\mu,\sigma^2\right)$ лежит в интервале $\mu\pm c \cdot \sigma$. Чему равно точное значение константы c? Округлите ответ до четырёх знаков после десятичной точки.

In [1]:
import scipy
from statsmodels.stats.weightstats import *

In [6]:
c = scipy.stats.norm.ppf(1 - 0.003/2) 
print(round(c, 4))

2.9677


Answer1 = 2.9677

5.
В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.

За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.

Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [11]:
round((189/11034) - (104/11037), 4)

0.0077

In [ ]:
Answer5 = 0.0077

6.
Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки.

In [12]:
from statsmodels.stats.proportion import proportion_confint

In [14]:
def proportions_confint_diff_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [15]:
aspirin = np.array([1] * 104 + [0] * (11037-104))
placebo = np.array([1] * 189 + [0] * (11034-189))

In [16]:
pl_asp_interval = proportions_confint_diff_ind(placebo, aspirin, alpha = 0.05)
pl_asp_interval

(0.004687750675049439, 0.010724297276960124)

Answer6 = 0.0107

7.
Продолжим анализировать данные эксперимента Гарвардской медицинской школы.

Для бернуллиевских случайных величин $X\sim Ber(p)$ часто вычисляют величину $\frac{p}{1-p}$, которая называется шансами (odds). Чтобы оценить шансы по выборке, вместо pp нужно подставить $\hat{p}$. Например, шансы инфаркта в контрольной группе, принимавшей плацебо, можно оценить как $\frac{\frac{189}{11034}}{1-\frac{189}{11034}} = \frac{189}{11034-189}\approx 0.0174$
Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [17]:
# шансы инфаркта при приеме аспирина
104 / (11037 - 104)

0.009512485136741973

In [46]:
# шансы инфаркта при приеме плацебо
189 / (11034 - 189)

0.017427385892116183

In [47]:
round(0.017427385892116183 / 0.009512485136741973, 4)

1.8321

Answer7 = 1.8321

8.
Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы получить в точности такой же доверительный интервал, как у нас:

составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, а потом все нули;

установите random seed=0;

сделайте по 1000 псевдовыборок из каждой группы пациентов с помощью функции get_bootstrap_samples.

In [21]:
aspirin

array([1, 1, 1, ..., 0, 0, 0])

In [22]:
placebo

array([1, 1, 1, ..., 0, 0, 0])

In [25]:
import pandas as pd
aspirin_df = pd.DataFrame(aspirin)
placebo_df = pd.DataFrame(placebo)

In [42]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [39]:
chance = lambda x : sum(x[x == 1])/(len(x)-sum(x[x == 1]))

In [48]:
np.random.seed(0)

aspirin_scores = list(map(chance, get_bootstrap_samples(aspirin, 1000)))
placebo_scores = list(map(chance, get_bootstrap_samples(placebo, 1000)))

delta_median_scores = list(map(lambda x: x[1] / x[0], zip(aspirin_scores, placebo_scores)))

print("95% confidence interval for the aspirin:",  stat_intervals(aspirin_scores, 0.05))
print("95% confidence interval for the placebo:",  stat_intervals(placebo_scores, 0.05))
print("95% confidence interval for the delta:",  stat_intervals(delta_median_scores, 0.05))

95% confidence interval for the aspirin: [0.00757714 0.0113626 ]
95% confidence interval for the placebo: [0.01499402 0.01996672]
95% confidence interval for the delta: [1.44419465 2.34321168]


Answer8 = 1.4442